# Tarea 1: Activaciones y pasada hacia adelante en una red neuronal <br/> CC6204 Deep Learning, Universidad de Chile  <br/> Hoja de respuestas
## Nombre: Gabriel Rojas
Fecha de entrega: 30 de agosto de 2021

In [ ]:
# Este notebook está pensado para correr en CoLaboratory. 
# Lo único imprescindible por importar es torch 
import torch

# Posiblemenete quieras instalar e importar ipdb para debuggear.
# Si es así, descomenta lo siguiente
# !pip install -q ipdb
# import ipdb

# Aqui instalamos la libreria de correccion del curso
!pip install "git+https://github.com/dccuchile/CC6204.git@master#egg=cc6204&subdirectory=autocorrect"

  Cloning https://github.com/dccuchile/CC6204.git (to revision master) to /tmp/pip-install-n3a_81es/cc6204_075ec6b5f7df4414a2638b065fe717fa
  Running command git clone -q https://github.com/dccuchile/CC6204.git /tmp/pip-install-n3a_81es/cc6204_075ec6b5f7df4414a2638b065fe717fa


In [ ]:
# importamos las herramientas del curso
from cc6204 import AutoCorrect, FailedTest

# ingresa el host y port que posteamos en u-cursos

corrector = AutoCorrect(host="cc6204.dcc.uchile.cl", port=443)

# anota el token que te daremos en u-cursos

token = "]ye/Ox;nsz"

Connection stablished


# Parte 1: Funciones de activación y función de salida

## 1a) Funciones de activación

In [ ]:
def sig(T):
  return torch.reciprocal(1 + torch.exp(-1 * T))

def tanh(T):
  E = torch.exp(T)
  e = torch.exp(-1 * T)
  return (E - e) * torch.reciprocal(E + e)

# Tu código acá
def relu(T):
  zero = torch.tensor([0], device=T.get_device())
  return torch.maximum(T, zero)

def swish(T, beta=1):
  return T * sig(beta * T)

def celu(T, alpha=1):
  zero = torch.tensor([0], device=T.get_device())
  return torch.maximum(zero, T) + torch.minimum(zero, alpha * (torch.exp(T / alpha) - 1))

In [ ]:
# Tests del API del curso
test_relu = corrector.get_test_data(homework=1, question="1a", test=1, token=token)
test_swish, swish_par = corrector.get_test_data(homework=1, question="1a", test=2, token=token)
test_celu, celu_par = corrector.get_test_data(homework=1, question="1a", test=3, token=token)

# probablemente quieras convertr los variables test_* a un tensor, ya que por defecto son listas

corrector.submit(homework=1, question="1a", test=1, token=token, answer=relu(torch.tensor(test_relu).to('cuda')))
corrector.submit(homework=1, question="1a", test=2, token=token, answer=swish(torch.tensor(test_swish).to('cuda'), swish_par))
corrector.submit(homework=1, question="1a", test=3, token=token, answer=celu(torch.tensor(test_celu).to('cuda'), celu_par))

Correct Test!
Correct Test!
Correct Test!


## 1b) Softmax

(La demostración puedes entregarla en otro archivo o incluirla directamente acá)

Demostracion: \\
Veamos que si a cada elemento se le resta el mismo valor $M$ entonces se tendra que
$$
s_i = \frac{e^{x_i-M}}{\sum_{j=1}^{n}e^{x_j-M}}
$$
lo que es igual a
$$
s_i = \frac{e^{x_i}e^{-M}}{\sum_{j=1}^{n}e^{x_j}e^{-M}}
$$
y si simplificamos se tiene que 
$$
s_i = \frac{e^{x_i}}{\sum_{j=1}^{n}e^{x_j}}
$$
Entonces queda demostrado que restarle el mismo valor a todos los elementos el resultado de softmax no varia.

In [ ]:
# Tu código acá

def softmax(T, dim=1, estable=True):
  if estable:
    T = T - torch.max(T)
  sum = torch.sum(torch.exp(T), dim=dim, keepdim=True)
  return torch.divide(torch.exp(T), sum)

In [ ]:
# Tests del API del curso
test_softmax, _dim = corrector.get_test_data(homework=1, question="1b", test=1, token=token)
corrector.submit(homework=1, question="1b", test=1, token=token, answer=softmax(torch.tensor(test_softmax).to('cuda'), dim=_dim))

Correct Test!


# Parte 2: Red neuronal y pasada hacia adelante (forward)

## 2a) Clase para red neuronal, 2b) Iterando por parametros, 2d) Pasada hacia adelante

In [ ]:
# Tu código debiera continuar así 

class FFNN(torch.nn.Module):
  def __init__(self, F, l_h, l_a, C, beta=1, alpha=1):
    super(FFNN, self).__init__()
    self.F = F
    dims = [F] + l_h
    self.W = torch.nn.ParameterList([torch.nn.Parameter(torch.rand(dims[x], dims[x+1])) for x in range(len(l_h))])
    self.b = torch.nn.ParameterList([torch.nn.Parameter(torch.rand(dims[x])) for x in range(1, len(dims))])
    self.U = torch.nn.Parameter(torch.rand(l_h[len(l_h)-1], C))
    self.c = torch.nn.Parameter(torch.rand(C))
    self.l_a = l_a
    self.beta = torch.nn.Parameter(torch.tensor([float(beta)]))
    self.alpha = torch.nn.Parameter(torch.tensor([float(alpha)]))

  def resumen(self):
    # usa self.parameters() o self.named_parameters()
    for param in self.named_parameters():
      print(param[0], "containing:", type(param[1].data), param[1].size())
  
  def forward(self, x):
    # Usa los parámetros y funciones de activación.
    # El valor de retorno debiera ser y = softmax(capa_de_salida).
    x.to('cuda')
    x = x.view(-1, self.F)
    for i in range(len(self.l_a)):
      if self.l_a[i] == swish:
        x = self.l_a[i](torch.mm(x, self.W[i]) + self.b[i], beta=float(self.beta))
      elif self.l_a[i] == celu:
        x = self.l_a[i](torch.mm(x, self.W[i]) + self.b[i], alpha=float(self.alpha))
      else:  
        x = self.l_a[i](torch.mm(x, self.W[i]) + self.b[i])
    x = torch.mm(x, self.U) + self.c
    return softmax(x)

## 2c) Moviendo los parámetros entre dispositivos

In [ ]:
# Tu código acá
red_neuronal = FFNN(300,[50,30],[relu,sig],10)
red_neuronal.to('cuda')
!nvidia-smi
red_neuronal.resumen()

Sat Sep  4 02:12:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    58W / 149W |    448MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Parte 3: Probando tu red con parámetros pre-entrenados para MNIST

In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

# Descarga y almacena el conjunto de prueba de MNIST.
dataset = MNIST('mnist', train=False, transform=ToTensor(), download=True)
print('Cantidad total de datos:',len(dataset))

Cantidad total de datos: 10000


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 3b) Cargando los parámetros pre-entrenados

In [ ]:
%%bash
# Este comando es util para descargar los datos
wget \
    --quiet \
    --no-clobber \
    --base https://raw.githubusercontent.com/dccuchile/CC6204/master/2020/tareas/tarea1/mnist_weights/ \
    --input-file - \
<<EOF
U.txt
W1.txt
W2.txt
b1.txt
b2.txt
c.txt
EOF

In [ ]:
# Tu código acá
from numpy import loadtxt
U = torch.from_numpy(loadtxt('U.txt')).float()
W1 = torch.from_numpy(loadtxt('W1.txt')).float()
W2 = torch.from_numpy(loadtxt('W2.txt')).float()
b1 = torch.from_numpy(loadtxt('b1.txt')).float()
b2 = torch.from_numpy(loadtxt('b2.txt')).float()
c = torch.from_numpy(loadtxt('c.txt')).float()

red = FFNN(784,[32,16],[relu,relu],10)
red.W[0] = torch.nn.Parameter(W1)
red.W[1] = torch.nn.Parameter(W2)
red.b[0] = torch.nn.Parameter(b1)
red.b[1] = torch.nn.Parameter(b2)
red.U = torch.nn.Parameter(U)
red.c = torch.nn.Parameter(c)
red.to('cuda')

red.resumen()

U containing: <class 'torch.Tensor'> torch.Size([16, 10])
c containing: <class 'torch.Tensor'> torch.Size([10])
beta containing: <class 'torch.Tensor'> torch.Size([1])
alpha containing: <class 'torch.Tensor'> torch.Size([1])
W.0 containing: <class 'torch.Tensor'> torch.Size([784, 32])
W.1 containing: <class 'torch.Tensor'> torch.Size([32, 16])
b.0 containing: <class 'torch.Tensor'> torch.Size([32])
b.1 containing: <class 'torch.Tensor'> torch.Size([16])


## 3c) Calcula la predicción de un ejemplo al azar

In [ ]:
# Tu código acá
import random
idx = random.randint(0,len(dataset))
T, l = dataset[idx]
out = red.forward(dataset[idx][0].to('cuda'))
pred = int(torch.argmax(out))
print("prediction:", pred, "\nanswer:", l)

prediction: 4 
answer: 4


## 3d) Pasando todos los ejemplos por la red con un `DataLoader`

In [ ]:
# Acá tu código
from torch.utils.data import DataLoader
def calcula_acierto(red, dataset, batch_size=100, device='cuda'):
  dataloader = DataLoader(dataset, batch_size=batch_size)
  count = 0.0
  correct = 0.0
  for x, y in dataloader:
    x = x.to('cuda')
    y = y.to('cuda')
    out = torch.argmax(red.forward(x.to('cuda')), dim=1)
    is_close = torch.isclose(out,y)
    correct += torch.sum(is_close)
    count += batch_size
  print("acierto: ", float(correct / count * 100), "%")

In [ ]:
calcula_acierto(red, dataset)

acierto:  96.12000274658203 %


### Corrección red

In [ ]:
# Tests del API del curso
from torch.utils.data import Subset
indices = corrector.get_test_data(homework=1, question="network", test=1, token=token)
test_set = Subset(dataset, indices)

# Modelo con los parámetros pre-entrenados para MNIST
your_network = red

# Montar el `test_set` en un tensor de (N, 28*28) usando DataLoader
X = list(DataLoader(test_set, batch_size=len(test_set)))[0][0].view(-1, 28*28)

# Almacenar el resultado en un puro tensor de (N,1)
result = torch.argmax(your_network(X.to('cuda')), dim=1)

corrector.submit(homework=1, question="network", test=1, token=token, answer=result)

Correct Test!


## 3e) Opcional: Muestra los casos en donde la red se equivoca

In [ ]:
# Acá tu código
dataloader = DataLoader(dataset, batch_size=1)
for x, y in dataloader:
  x = x.to('cuda')
  y = y.to('cuda')
  out = torch.argmax(red.forward(x.to('cuda')), dim=1)
  is_close = torch.isclose(out,y)
  for n in range(len(is_close)):
    if not is_close[n]:
      print("prediction:", int(out[n]), "answer:", int(y[n]))

prediction: 6 answer: 5
prediction: 8 answer: 9
prediction: 2 answer: 4
prediction: 0 answer: 6
prediction: 7 answer: 9
prediction: 7 answer: 2
prediction: 3 answer: 5
prediction: 7 answer: 3
prediction: 3 answer: 5
prediction: 2 answer: 8
prediction: 3 answer: 8
prediction: 1 answer: 7
prediction: 2 answer: 3
prediction: 2 answer: 8
prediction: 3 answer: 8
prediction: 8 answer: 1
prediction: 9 answer: 3
prediction: 8 answer: 2
prediction: 3 answer: 7
prediction: 4 answer: 8
prediction: 9 answer: 4
prediction: 6 answer: 0
prediction: 8 answer: 5
prediction: 9 answer: 4
prediction: 9 answer: 8
prediction: 2 answer: 7
prediction: 9 answer: 7
prediction: 9 answer: 4
prediction: 4 answer: 5
prediction: 2 answer: 1
prediction: 5 answer: 6
prediction: 8 answer: 5
prediction: 9 answer: 7
prediction: 2 answer: 6
prediction: 9 answer: 3
prediction: 8 answer: 9
prediction: 6 answer: 4
prediction: 8 answer: 3
prediction: 2 answer: 3
prediction: 9 answer: 7
prediction: 6 answer: 4
prediction: 1 an

## 3d) Opcional: Crea tus propios ejemplos de dígitos para clasificar

In [ ]:
# Acá tu código